In [ ]:
import dlib
import cv2
import numpy as np
import face_recognition
import pandas as pd

In [ ]:
train_data=pd.read_csv(r"train\annos.csv")
train_data.insert(train_data.shape[1],'EuDist',-1)
train_data.insert(train_data.shape[1],'SimDist',-1)


In [ ]:
train_data.at[ 1, "EuDist"]=1

In [ ]:
image = face_recognition.load_image_file(r"D:\1mylearningdata\1mphil\Competition\NCCCU2022\1my\train\data\452\a.jpg")
image = cv2.fastNlMeansDenoisingColored(image, None, 30, 30, 7, 21)

face_locations = face_recognition.face_locations(image)

img = image[face_locations[0][0]:face_locations[0][2], face_locations[0][3]:face_locations[0][1]]
face_encoding = face_recognition.face_encodings(image, face_locations)[0]

In [ ]:
a = np.asarray([ [1,2,3], [4,5,6], [7,8,9] ])
pd.DataFrame(a).to_csv('sample.csv')

In [1]:
import dlib
import cv2
import numpy as np
import face_recognition
import pandas as pd
import os


def getFaceEncoding(src):
    image = face_recognition.load_image_file(src)
    image = cv2.medianBlur(image, 3)
    image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    face_locations = face_recognition.face_locations(image)
    if face_locations == []:
        return np.array([])
    img = image[face_locations[0][0]:face_locations[0][2], face_locations[0][3]:face_locations[0][1]]
    face_encoding = face_recognition.face_encodings(image, face_locations)[0]
    return face_encoding


def getEuDist(img_encoding1, img_encoding2):
    img_encoding1 = np.array(img_encoding1)
    img_encoding2 = np.array(img_encoding2)
    diff = np.subtract(img_encoding1, img_encoding2)
    dist = np.sqrt(np.sum(np.square(diff)))
    return dist


def getSimDist(img_encoding1, img_encoding2):
    img_encoding1 = np.array(img_encoding1)
    img_encoding2 = np.array(img_encoding2)
    dist = np.linalg.norm(img_encoding1 - img_encoding2)
    sim = 1.0 / (1.0 + dist)
    return sim


id_imgGroup=[]
train_data=pd.read_csv(r"train\annos.csv")
train_data.insert(train_data.shape[1],'EuDist',-1)
train_data.insert(train_data.shape[1],'SimDist',-1)
train_path = "train\data"
file = os.walk(train_path)
for sub_path, sub_dir_list, sub_file_list in file:
    for sub_dir_name in sub_dir_list:
        sub_file = os.walk(os.path.join(sub_path, sub_dir_name))

        for sub2_path, sub2_dir_list, sub2_file_list in sub_file:
            img_encoding1 = getFaceEncoding(os.path.join(sub2_path, sub2_file_list[0]))
            img_encoding2 = getFaceEncoding(os.path.join(sub2_path, sub2_file_list[1]))
            if (img_encoding1.size == 0) or (img_encoding2.size == 0):
                continue
            else:
                print(sub_dir_name)
                eu_dist = getEuDist(img_encoding1, img_encoding2)
                sim_dist = getSimDist(img_encoding1, img_encoding2)
                train_data.at[int(sub_dir_name),"EuDist"]=eu_dist
                train_data.at[int(sub_dir_name),"SimDist"]=sim_dist
                # print(os.path.join(sub2_path, sub2_file_name))


0
10
102
103
104
105
106
108
11
113
114
117
119
12
121
122
124
125
127
128
13
130
131
133
137
138
139
142
143
145
147
148
15
150
152
153
154
155
156
160
166
168
17
171
172
173
175
176
177
178
181
183
185
192
195
196
197
198
199
2
20
201
204
205
21
211
213
215
216
218
219
22
220
223
226
227
228
23
231
232
234
235
236
237
239
24
240
241
243
244
246
248
249
25
250
253
254
255
257
258
259
261
263
264
265
266
267
268
269
27
270
274
275
276
277
278
279
28
280
281
282
283
284
285
286
287
288
289
29
291
294
295
296
297
299
3
30
300
302
303
304
306
308
309
31
310
311
313
314
316
317
32
321
322
324
326
327
328
330
331
334
335
336
339
340
342
343
344
345
347
348
350
351
352
353
354
355
356
357
359
360
363
365
366
367
369
370
371
372
374
376
377
378
38
380
382
383
385
387
388
389
390
392
393
395
396
400
401
402
403
404
406
407
409
415
416
417
419
42
420
421
422
423
424
425
427
429
43
430
432
433
434
435
436
437
438
439
44
440
441
442
443
444
445
446
447
448
454
457
459
46
460
461
462
463
464
465
4

In [2]:
train_data.to_csv("train_data2.csv",index_label="id")